Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import torch 
# Libraries for processing sounds
import librosa
from IPython.display import Audio
from torch.utils.data import Dataset
import random

change the .csv so the path is correct and add row for the class

In [2]:
def change_path(path, new_path):
    train_csv = pd.read_csv(path)
    train_csv['path']=new_path
    train_csv['class'] = train_csv['name'].apply(lambda x: x.split('_')[0])
    train_csv.to_csv('data.csv')
    
    
    
path = 'C:/Users/Lorena/Documents/Uni/25 SoSe/Deep Learning/DL-mine/Animal_Sound.csv'
new_path = 'C:/Users/Lorena/Documents/Uni/25 SoSe/Deep Learning/DL-mine/Animal-Soundprepros'
change_path(path, new_path=new_path)

data_path = 'C:/Users/Lorena/Documents/Uni/25 SoSe/Deep Learning/DL-mine/data.csv'
train_csv = pd.read_csv(data_path)

In [3]:
print(train_csv.head())

   Unnamed: 0         name                                               path  \
0           0   Lion_1.wav  C:/Users/Lorena/Documents/Uni/25 SoSe/Deep Lea...   
1           1  Lion_10.wav  C:/Users/Lorena/Documents/Uni/25 SoSe/Deep Lea...   
2           2  Lion_11.wav  C:/Users/Lorena/Documents/Uni/25 SoSe/Deep Lea...   
3           3  Lion_12.wav  C:/Users/Lorena/Documents/Uni/25 SoSe/Deep Lea...   
4           4  Lion_13.wav  C:/Users/Lorena/Documents/Uni/25 SoSe/Deep Lea...   

   channels  sample_width  frame_rate  nframes  duration  size class  
0         2             2       22050    99296  4.503220  4096  Lion  
1         2             2       11025    75964  6.890159  4096  Lion  
2         1             1       22000    31744  1.442909  4096  Lion  
3         1             2       11025    87108  7.900952  4096  Lion  
4         1             1       11025    93972  8.523537  4096  Lion  


Bring into form so it can be used in ML

PyTorch Website:
    https://docs.pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [ ]:
def extract_mel_spectrogram(file_path, sr=22050, n_mels=128):
    y, sr = librosa.load(file_path, sr=sr)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    return mel_db

Define new Datset, so we get a training and validation Dataset from .wav files

In [ ]:
class AnimalSoundDataset(Dataset):
    def __init__(self, root_dir, transform=None, split='train', split_ratio=0.8, seed=42):
        self.root_dir = root_dir
        self.classes = sorted(os.listdir(root_dir))
        self.transform = transform

        all_paths = []
        all_labels = []

        for idx, class_name in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_name)
            for file_name in os.listdir(class_path):
                if file_name.endswith(".wav"):
                    all_paths.append(os.path.join(class_path, file_name))
                    all_labels.append(idx)

        # Shuffle and split
        combined = list(zip(all_paths, all_labels))
        random.seed(seed)
        random.shuffle(combined)
        split_point = int(len(combined) * split_ratio)

        if split == 'train':
            selected = combined[:split_point]
        elif split == 'val':
            selected = combined[split_point:]
        else:
            raise ValueError("split must be 'train' or 'val'")

        self.file_paths, self.labels = zip(*selected) if selected else ([], [])

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        mel = extract_mel_spectrogram(self.file_paths[idx])  # [n_mels, time]
        #mel = pad_or_trim(mel, target_width=400)
        mel = torch.tensor(mel, dtype=torch.float32).unsqueeze(0)  # [1, H, W]
        label = self.labels[idx]
        return mel, label
    
    
        
    def get_class(self,idx):
        label = self.labels[idx]
        return self.classes[label]
    
    def visualize(self,n):
    #print(dataset[n][0].squeeze(0))
        plt.figure(figsize=(16,6))
        librosa.display.specshow(
                            self[n][0].squeeze(0).numpy(),
                            x_axis="time",
                            y_axis="mel")
        plt.colorbar()

    def play(self,n):
        path = self.file_paths[n]
        #print(path)
        x, Fs = librosa.load(path, sr=None)
        label = self.get_class(n)
        print('Class: {}'.format(label))
        return Audio(x, rate=Fs)

How you use the dataset

In [ ]:
path_parent_project = os.getcwd() #current walk directory
dataset_image_path = path_parent_project + '\\Animal-Soundprepros\\'

dataset_train = AnimalSoundDataset(dataset_image_path, split='train', split_ratio=0.8, seed=42)
dataset_val = AnimalSoundDataset(dataset_image_path, split='val', split_ratio=0.8, seed=42)

In [ ]:
dataset_train.play(2)

Class: Dog
